In [1]:
%load_ext autoreload
%autoreload 2
from train import NNDynamicModel, MPCcontroller
import tensorflow as tf
import numpy as np
import json

/Users/loulinhui/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
regularizer = tf.contrib.layers.l2_regularizer(scale=1e-5)
activation = tf.nn.relu
output_activation = None
learning_rate = 0.003
batch_size = 128

In [3]:
# load data
with open('./sample.json', 'r') as f:
    data = json.loads(f.read())
    
print('loaded %d data.' % (len(data)))


loaded 8000 data.


In [4]:
obs = [d[0] for d in data]
normalization = [np.min(obs, axis=0), np.max(obs, axis=0), np.mean(obs, axis=0)]
print(normalization)

session = tf.Session()

[array([  0.   , -50.   ,  90.   ,  17.   ,   6.166]), array([  0.   , 625.   , 105.   ,  51.   ,   6.657]), array([  0.        , 424.27775   ,  97.558125  ,  29.69      ,
         6.21954263])]


In [5]:
model = NNDynamicModel(
    name='test',
    n_layers=3,
    size=500,
    activation=activation,
    output_activation=output_activation,
    kernel_regularizer=regularizer,
    normalization=normalization,
    batch_size=batch_size,
    iterations=40,
    learning_rate=learning_rate,
    sess=session
)

In [6]:
init = tf.global_variables_initializer()
session.run(init)

In [7]:
model.fit(data)

on iter_step 0, loss = 0.1105976
on iter_step 1, loss = 0.0854701
on iter_step 2, loss = 0.0821772
on iter_step 3, loss = 0.0822450
on iter_step 4, loss = 0.0805927
on iter_step 5, loss = 0.0779966
on iter_step 6, loss = 0.0777574
on iter_step 7, loss = 0.0762136
on iter_step 8, loss = 0.0751460
on iter_step 9, loss = 0.0757106
on iter_step 10, loss = 0.0751115
on iter_step 11, loss = 0.0757109
on iter_step 12, loss = 0.0741578
on iter_step 13, loss = 0.0733339
on iter_step 14, loss = 0.0733218
on iter_step 15, loss = 0.0722440
on iter_step 16, loss = 0.0725285
on iter_step 17, loss = 0.0715083
on iter_step 18, loss = 0.0729600
on iter_step 19, loss = 0.0713228
on iter_step 20, loss = 0.0709960
on iter_step 21, loss = 0.0716361
on iter_step 22, loss = 0.0717889
on iter_step 23, loss = 0.0712208
on iter_step 24, loss = 0.0707964
on iter_step 25, loss = 0.0710603
on iter_step 26, loss = 0.0709844
on iter_step 27, loss = 0.0699346
on iter_step 28, loss = 0.0700500
on iter_step 29, loss = 

In [10]:
mpc = MPCcontroller(dyn_model=model) 

In [18]:
nd = [d for d in data if d[-1] < 0]
obs, action, nobs, reward = nd[2]
print(model.predict([obs], [action]))
print(data[0])
action = mpc.get_action(obs)
print(action)

(array([[-1.8611588e-03,  8.2346596e+01,  8.9301704e+01,  2.4753624e+01,
         6.2678785e+00]], dtype=float32), array([-1.0655589], dtype=float32))
[[0.0, 617.0, 105.0, 17.0, 6.477000000000159], [2.0, 0.33731651256761297], [0.0, 479.0, 105.0, 17.0, 6.525000000000175], 1]
[0.         0.24290873] -0.6701787
[0.         0.16634088] -0.2916547
[0.        0.8814499] -1.0609406
[0.        0.0295204] 0.31493282
[0.        0.0549637] 0.40419942
[0.         0.39289889] -0.9561287
[0.         0.42682002] -0.9831791
[0.         0.85944618] -1.0579916
[0.         0.63408632] -1.0280226
[0.         0.79720459] -1.0502659
[0.         0.61501808] -1.0263752
[0.         0.40944158] -0.96940124
[0.         0.11447888] 0.21568504
[0.         0.15532578] -0.19591348
[0.         0.82511186] -1.0536741
[0.         0.00076467] 0.23675767
[0.         0.42785602] -0.9839648
[0.         0.09884676] 0.36462563
[0.         0.45768539] -0.9996028
[0.         0.25365702] -0.70123196
[1.         0.81047787] 0.79